**Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import Libraries**

In [ ]:
import os
import re
import sys
import numpy as np
from numpy import mean,std
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import utils
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics

import itertools
from itertools import chain, product
from itertools import zip_longest

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

import seaborn as sns

**Load Dataset**

In [ ]:
Train_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Train.csv')
Test_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Test.csv')
Val_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Val.csv')

In [ ]:
Train_data.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
Val_data.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,fake
1,2,11 out of 13 people (from the Diamond Princess...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [ ]:
Test_data.head()

,id,tweet
0,1,Our daily update is published. States reported...
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He W...
3,4,States reported 630 deaths. We are still seein...
4,5,This is the sixth time a global health emergen...


In [ ]:
Train_data.shape, Val_data.shape, Test_data.shape

((6420, 3), (2140, 3), (2140, 2))

**Text Preprocessing**

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 


def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
Train_data['cleanText'] = Train_data['tweet'].map(lambda s:preprocess(s))
Val_data['cleanText'] = Val_data['tweet'].map(lambda s:preprocess(s))
Test_data['cleanText'] = Test_data['tweet'].map(lambda s:preprocess(s))

In [ ]:
Train_data.head()

,id,tweet,label,cleanText
0,1,The CDC currently reports 99031 deaths. In gen...,real,cdc currently reports deaths general discrepan...
1,2,States reported 1121 deaths a small rise from ...,real,states reported deaths small rise last tuesday...
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,politically correct woman almost uses pandemic...
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,indiafightscorona covid testing laboratories i...
4,5,Populous states can generate large case counts...,real,populous states generate large case counts loo...


In [ ]:
Val_data.head()

,id,tweet,label,cleanText
0,1,Chinese converting to Islam after realising th...,fake,chinese converting islam realising muslim affe...
1,2,11 out of 13 people (from the Diamond Princess...,fake,people diamond princess cruise ship intially t...
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake,covid caused bacterium virus treated aspirin
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake,mike pence rnc speech praises donald trump cov...
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real,sky edconwaysky explains latest covid data gov...


In [ ]:
Test_data.head()

,id,tweet,cleanText
0,1,Our daily update is published. States reported...,daily update published states reported tests n...
1,2,Alfalfa is the only cure for COVID-19.,alfalfa cure covid
2,3,President Trump Asked What He Would Do If He W...,president trump asked would catch coronavirus ...
3,4,States reported 630 deaths. We are still seein...,states reported deaths still seeing solid nati...
4,5,This is the sixth time a global health emergen...,sixth time global health emergency declared in...


In [ ]:
Train_data.shape, Val_data.shape, Test_data.shape

((6420, 4), (2140, 4), (2140, 3))

**Tagging Document and Splitting Dataset**

In [ ]:
X_train = Train_data['cleanText'][0:500]
y_train = Train_data['label'][0:500]

X_test = Val_data['cleanText'][0:500]
y_test = Val_data['label'][0:500]

In [ ]:
from gensim.models import doc2vec

def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the complaint narrative.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.TaggedDocument(v.split(), [label]))
    return labeled

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test

In [ ]:
len(all_data)

1000

In [ ]:
all_data[:5]

[TaggedDocument(words=['cdc', 'currently', 'reports', 'deaths', 'general', 'discrepancies', 'death', 'counts', 'different', 'sources', 'small', 'explicable', 'death', 'toll', 'stands', 'roughly', 'people', 'today'], tags=['Train_0']),
 TaggedDocument(words=['states', 'reported', 'deaths', 'small', 'rise', 'last', 'tuesday', 'southern', 'states', 'reported', 'deaths'], tags=['Train_1']),
 TaggedDocument(words=['politically', 'correct', 'woman', 'almost', 'uses', 'pandemic', 'excuse', 'reuse', 'plastic', 'bag', 'coronavirus', 'nashville'], tags=['Train_2']),
 TaggedDocument(words=['indiafightscorona', 'covid', 'testing', 'laboratories', 'india', 'august', 'tests', 'done', 'profbhargava', 'icmrdelhi', 'staysafe', 'indiawillwin'], tags=['Train_3']),
 TaggedDocument(words=['populous', 'states', 'generate', 'large', 'case', 'counts', 'look', 'new', 'cases', 'per', 'million', 'today', 'smaller', 'states', 'showing', 'cases', 'per', 'million', 'california', 'texas'], tags=['Train_4'])]

**DBOW Vectors**

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

100%|██████████| 1000/1000 [00:00<00:00, 391223.21it/s]


In [ ]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 1000/1000 [00:00<00:00, 492346.99it/s]


CPU times: user 3.1 s, sys: 307 ms, total: 3.41 s
Wall time: 2.58 s


In [ ]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors

In [ ]:
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')  #X-train Vectors
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')     #X-test Vectors

**Model Evaluation**

In [ ]:
dict_classifiers = {
    "Nearest Neighbors":KNeighborsClassifier(3),
    "LogisticRegression":LogisticRegression(),
    #"NuSVC":NuSVC(probability=True),
    "Decision Tree": DecisionTreeClassifier(max_depth=5),
    "Random Forest": RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    "MLP": MLPClassifier(alpha=1, max_iter=1000),
    "AdaBoost":AdaBoostClassifier(),
    "GaussianNB":GaussianNB(),
    "LinearDiscriminant":LinearDiscriminantAnalysis(),
    "QuadraticDiscriminant":QuadraticDiscriminantAnalysis(),
    "GradientBoost":GradientBoostingClassifier(),
    #"MultinomialNB":MultinomialNB(),
    "SGD":SGDClassifier(),
    "LGBM":LGBMClassifier(),
    #"XGB":XGBClassifier(),
    "Gradient Process Classifier":GaussianProcessClassifier(1.0 * RBF(1.0)),
    "Support Vector Machine 1":SVC(kernel="linear", C=0.025),
    "Support Vector Machine 2":SVC(gamma=2, C=1)
}

In [ ]:
def evaluation_matrix(model, testX, testy):
    # predict probabilities for test set
    yhat_probs = model.predict(testX)
    #print(yhat_probs)
    # predict crisp classes for test set
    #yhat_classes = model.predict_classes(testX)
    # reduce to 1d array
    #yhat_probs = yhat_probs[:, 0]
    #print(yhat_probs)
    #yhat_classes = yhat_classes
    yhat_classes = yhat_probs
    #accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(testy, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(testy, yhat_classes, average='weighted')
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(testy, yhat_classes, average='weighted')
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(testy, yhat_classes, average='weighted')
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(testy, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    #auc = roc_auc_score(testy, yhat_probs, multi_class = 'ovr')
    #print('ROC AUC: %f' % auc)
    #confusion matrix
    matrix = confusion_matrix(testy, yhat_classes)
    print(matrix)
    return accuracy, precision, recall, f1, kappa

In [ ]:
def batch_classify(dict_classifiers, train_vectors_dbow, y_train, test_vectors_dbow, y_test):
    no_classifiers = len(dict_classifiers)
    dict_super_models = {}

    for classifier_name,classifier in list(dict_classifiers.items())[:no_classifiers]:
        print(classifier_name)
        #classifier.fit(X_train, y_train)
        classifier.fit(train_vectors_dbow, y_train)

        #train_score = classifier.score(X_train, y_train)
        train_score = classifier.score(train_vectors_dbow, y_train)

        #test_score = classifier.score(X_test, y_test)
        test_score = classifier.score(test_vectors_dbow, y_test)

        #accuracy, precision, recall, f1, kappa = evaluation_matrix(classifier, X_test, y_test)
        accuracy, precision, recall, f1, kappa = evaluation_matrix(classifier, test_vectors_dbow, y_test)

        #y_pred = classifier.predict(X_test)
        y_pred = classifier.predict(test_vectors_dbow)

        print('train score:{%0.2f} test_score:{%0.2f}' %(train_score, test_score))
        dict_super_models[classifier_name] = {'Supervised': classifier_name, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1, 'Kappa': kappa}   

    return dict_super_models

In [ ]:
dict_super_models = batch_classify(dict_classifiers, train_vectors_dbow, y_train, test_vectors_dbow, y_test)

Nearest Neighbors
Accuracy: 0.800000
Precision: 0.805407
Recall: 0.800000
F1 score: 0.799808
Cohens kappa: 0.601333
[[204  35]
 [ 65 196]]
train score:{0.90} test_score:{0.80}
LogisticRegression
Accuracy: 0.844000
Precision: 0.844363
Recall: 0.844000
F1 score: 0.844060
Cohens kappa: 0.687725
[[203  36]
 [ 42 219]]
train score:{0.88} test_score:{0.84}
Decision Tree
Accuracy: 0.780000
Precision: 0.780835
Recall: 0.780000
F1 score: 0.780106
Cohens kappa: 0.559923
[[189  50]
 [ 60 201]]
train score:{0.92} test_score:{0.78}
Random Forest
Accuracy: 0.794000
Precision: 0.795024
Recall: 0.794000
F1 score: 0.793354
Cohens kappa: 0.585813
[[178  61]
 [ 42 219]]
train score:{0.90} test_score:{0.79}
MLP
Accuracy: 0.846000
Precision: 0.846003
Recall: 0.846000
F1 score: 0.845917
Cohens kappa: 0.691130
[[198  41]
 [ 36 225]]
train score:{0.88} test_score:{0.85}
AdaBoost
Accuracy: 0.810000
Precision: 0.810621
Recall: 0.810000
F1 score: 0.809576
Cohens kappa: 0.618253
[[184  55]
 [ 40 221]]
train score

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Accuracy: 0.826000
Precision: 0.826488
Recall: 0.826000
F1 score: 0.825682
Cohens kappa: 0.650524
[[189  50]
 [ 37 224]]
train score:{1.00} test_score:{0.83}
SGD
Accuracy: 0.796000
Precision: 0.817616
Recall: 0.796000
F1 score: 0.790839
Cohens kappa: 0.586247
[[154  85]
 [ 17 244]]
train score:{0.87} test_score:{0.80}
LGBM
Accuracy: 0.838000
Precision: 0.838183
Recall: 0.838000
F1 score: 0.837819
Cohens kappa: 0.674855
[[194  45]
 [ 36 225]]
train score:{1.00} test_score:{0.84}
Gradient Process Classifier
Accuracy: 0.856000
Precision: 0.855972
Recall: 0.856000
F1 score: 0.855972
Cohens kappa: 0.711340
[[202  37]
 [ 35 226]]
train score:{0.89} test_score:{0.86}
Support Vector Machine 1
Accuracy: 0.824000
Precision: 0.831106
Recall: 0.824000
F1 score: 0.823713
Cohens kappa: 0.649419
[[212  27]
 [ 61 200]]
train score:{0.84} test_score:{0.82}
Support Vector Machine 2
Accuracy: 0.774000
Precision: 0.781462
Recall: 0.774000
F1 score: 0.771225
Cohens kappa: 0.543501
[[160  79]
 [ 34 227]]
tr

In [ ]:
def display_dict_models(dict_models, sort_by='Accuracy'):
#def display_dict_models(dict_models, sort_by='train_score'):
    cls = [key for key in dict_models.keys()]
    accuracy = [dict_models[key]['Accuracy'] for key in cls]
    precision = [dict_models[key]['Precision'] for key in cls]
    recall = [dict_models[key]['Recall'] for key in cls]
    f1 = [dict_models[key]['F1'] for key in cls]
    kappa = [dict_models[key]['Kappa'] for key in cls]
    #training_t = [dict_models[key]['train_time'] for key in cls]
    
    #df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),6)), columns = ['classifier', 'Accuracy', 'Precision', 'Recall', 'F1', 'Kappa'])
    # df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),3)), columns = ['classifier', 'train_score', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'Accuracy'] = accuracy[ii]
        df_.loc[ii, 'Precision'] = precision[ii]
        df_.loc[ii, 'Recall'] = recall[ii]
        df_.loc[ii, 'F1'] = f1[ii]
        df_.loc[ii, 'Kappa'] = kappa[ii]
       
        #df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))
    return df_

In [ ]:
df = display_dict_models(dict_super_models)

,classifier,Accuracy,Precision,Recall,F1,Kappa
12,Gradient Process Classifier,0.856,0.855972,0.856,0.855972,0.711340
4,MLP,0.846,0.846003,0.846,0.845917,0.691130
1,LogisticRegression,0.844,0.844363,0.844,0.844060,0.687725
11,LGBM,0.838,0.838183,0.838,0.837819,0.674855
9,GradientBoost,0.826,0.826488,0.826,0.825682,0.650524
13,Support Vector Machine 1,0.824,0.831106,0.824,0.823713,0.649419
5,AdaBoost,0.810,0.810621,0.810,0.809576,0.618253
0,Nearest Neighbors,0.800,0.805407,0.800,0.799808,0.601333
10,SGD,0.796,0.817616,0.796,0.790839,0.586247
3,Random Forest,0.794,0.795024,0.794,0.793354,0.585813


In [ ]:
print(df.to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
                  classifier &  Accuracy &  Precision &  Recall &        F1 &     Kappa \\
\midrule
           Nearest Neighbors &     0.800 &   0.805407 &   0.800 &  0.799808 &  0.601333 \\
          LogisticRegression &     0.844 &   0.844363 &   0.844 &  0.844060 &  0.687725 \\
               Decision Tree &     0.780 &   0.780835 &   0.780 &  0.780106 &  0.559923 \\
               Random Forest &     0.794 &   0.795024 &   0.794 &  0.793354 &  0.585813 \\
                         MLP &     0.846 &   0.846003 &   0.846 &  0.845917 &  0.691130 \\
                    AdaBoost &     0.810 &   0.810621 &   0.810 &  0.809576 &  0.618253 \\
                  GaussianNB &     0.790 &   0.798607 &   0.790 &  0.789442 &  0.582060 \\
          LinearDiscriminant &     0.738 &   0.739886 &   0.738 &  0.738089 &  0.476553 \\
       QuadraticDiscriminant &     0.538 &   0.583080 &   0.538 &  0.493180 &  0.099766 \\
               GradientBoost &     0.826 &   0.8